<a href="https://colab.research.google.com/github/nhs-pycom/coding-club-trashPanda/blob/main/coding-club_complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Welcome to NHSX Coding Club

## Lesson One: trashPanda

### Pandas for excel

In [ ]:
# clone coding club repo to google colab
!git clone -l -s git://github.com/nhs-pycom/coding-club-trashPanda.git cloned-repo
%cd cloned-repo
!ls

In [ ]:
import pandas as pd
pd.set_option("display.max.columns", None) # show all columns in dataframe

# read an excel file from repo /data/ folder
path = '/content/cloned-repo/data/tabular structure/Q1 Returns/Jan 2021.xlsx'
df = pd.read_excel(path)
df.head()

In [ ]:
# specify which sheet to load and index column number (remember index starts at 0 in python)
df = pd.read_excel(path, sheet_name=0, index_col=0)
df.head()

In [ ]:
df.info()

In [ ]:
# basic transformations
sort_population = df.sort_values(['Population:'], ascending=False)
sort_population['Population:']

In [ ]:
import matplotlib.pyplot as plt

# bar chart
sort_population['Population:'].plot(kind="barh")
plt.show()

In [ ]:
# discriptive statistics
sort_population['Population:'].describe()

In [ ]:
# parse excel file as object
excel_file = pd.ExcelFile(path)
excel_file.sheet_names

In [ ]:
# parse sheet in excel file from sheet names list
excel_file.parse(excel_file.sheet_names[0]).head()

In [ ]:
# loop through sheets and concatanate them together
temp_list = []
for sheet in excel_file.sheet_names:
   temp_list.append(excel_file.parse(sheet))
all_df = pd.concat(temp_list)
all_df.reset_index(drop=True, inplace=True) # reset index
all_df.tail()

In [ ]:
import os

path = "/content/cloned-repo/data/tabular structure/"
my_filenames = [
    os.path.join(root, name) # join root folder and name of file
    for root, dirs, files in os.walk(path) # for all files and directories in path
    for name in files
    if name.endswith((".xlsx")) # that end with .xlsx (excel files)
]
my_filenames

In [ ]:
from fnmatch import fnmatch

temp_list = []

for filename in my_filenames:
    excel_file = pd.ExcelFile(filename)
    # filter out non-data sheets by calling those with 'CCG' in name
    sheet_list = [sheet for sheet in excel_file.sheet_names if fnmatch(sheet, "*CCG*")]
    for sheet in excel_file.sheet_names:
      temp_list.append(excel_file.parse(sheet)) # parse each sheet into list
all_df = pd.concat(temp_list)
all_df.reset_index(drop=True, inplace=True) # reset index
all_df.tail()

In [ ]:
temp_list = []

for filename in my_filenames:
    excel_file = pd.ExcelFile(filename)
    # filter out non-data sheets by calling those with 'CCG' in name
    sheet_list = [sheet for sheet in excel_file.sheet_names if fnmatch(sheet, "*CCG*")]
    for sheet in excel_file.sheet_names:
      temp_list.append(excel_file.parse(sheet)) # parse each sheet into list
df = pd.concat(temp_list)
df.reset_index(drop=True, inplace=True) # reset index
df

In [ ]:
df.to_csv(r'data/output.csv')

# trashPanda

In [ ]:
from openpyxl import load_workbook

path = '/content/cloned-repo/data/poor structure/Q1 Returns/Jan 2021.xlsx'
# Load in the workbook
wb = load_workbook(path)

# Get sheet names
print(wb.sheetnames)

In [ ]:
from datetime import datetime

# last modified datex
time_stamp = os.path.getmtime(path)
print(time_stamp)

In [ ]:
# last modified date
mod_date = datetime.fromtimestamp(time_stamp).strftime("%Y-%m-%d %H:%M:%S")
print(mod_date)

In [ ]:
# trashPanda function
import os
import pandas as pd
from openpyxl import load_workbook
from datetime import datetime
from fnmatch import fnmatch


def trashPanda(path):
    """Searches directories for excel files, extracts data and returns a structured pandas dataframe."""
    my_filenames = [
        os.path.join(root, name)
        for root, dirs, files in os.walk(path)
        for name in files
        if name.endswith((".xlsx"))
    ]
    df = pd.DataFrame(
        columns=[
            "file",
            "mod_date",
            "sheet",
            "Org name",
            "Area Name",
            "Completed by",
            "Date completed",
            "Population",
            "Number of Records",
            "Number of Users",
            "Number of Views",
            "Number of Deployments",
            "Number of Unique Users",
            "Name of Errors",
        ]
    )
    for filename in my_filenames:
        wb = load_workbook(filename)
        sheet_list = [
            sheet for sheet in wb.sheetnames if fnmatch(sheet, "*CCG*")
        ]
        # last modified date
        mod_date = datetime.fromtimestamp(os.path.getmtime(filename)).strftime(
            "%Y-%m-%d"
        )
        for sheet in sheet_list:
            df = df.append(
                {
                    "file": filename,
                    "mod_date": mod_date,
                    "sheet": wb[sheet].title,
                    "Org name": wb[sheet]["B1"].value,
                    "Area Name": wb[sheet]["D1"].value,
                    "Completed by": wb[sheet]["B4"].value,
                    "Date completed": wb[sheet]["D4"].value,
                    "Population": wb[sheet]["B2"].value,
                    "Number of Records": wb[sheet]["D2"].value,
                    "Number of Users": wb[sheet]["B3"].value,
                    "Number of Views": wb[sheet]["D3"].value,
                    "Number of Deployments": wb[sheet]["B5"].value,
                    "Number of Unique Users": wb[sheet]["B6"].value,
                    "Number of Errors": wb[sheet]["D6"].value,
                },
                ignore_index=True,
            )
    return df

In [ ]:
# call function
path = "/content/cloned-repo/data/poor structure/"
df = trashPanda(path)
df.to_csv(r'data/output.csv')

# Fuzzywuzzy

In [ ]:
!pip install fuzzywuzzy
!pip install -U PyYAML

In [ ]:
import os
import yaml
import pandas as pd
from openpyxl import load_workbook
from datetime import datetime
from fnmatch import fnmatch
from fuzzywuzzy import process
from fuzzywuzzy import fuzz

config = 'config.yaml'
with open(config, "r") as yamlfile:
    configs = yaml.load(yamlfile, Loader=yaml.FullLoader)

path = "data/poor structure"
df = pd.DataFrame(columns=configs['Column Names'])
dictionary = {}
limit = 1   # Number of results to return
cutoff = 80 # % similarity

my_filenames = [
    os.path.join(root, name)
    for root, dirs, files in os.walk(path)
    for name in files
    if name.endswith((".xlsx"))
]
# loop files
for filename in my_filenames:
    wb = load_workbook(filename)
    dictionary["file"] = filename
    sheet_list = [sheet for sheet in wb.sheetnames if fnmatch(sheet, configs["Sheet"])]
    # last modified date
    mod_date = datetime.fromtimestamp(os.path.getmtime(filename))
    dictionary["Modified date"] = mod_date
    # loop sheets
    for sheet in sheet_list:
        dictionary["sheet"] = sheet
        df_sheet = pd.read_excel(
            filename,
            sheet_name=sheet,
            engine="openpyxl",
            index_col=None,
            header=None,
            nrows=configs["Rows"],
        )
        for row in list(range(0, configs["Rows"])):
            for name in configs['Column Names']:
                match = process.extractBests(
                    name,
                    df_sheet.iloc[row],
                    limit=limit,
                    scorer=fuzz.token_sort_ratio,
                    score_cutoff=cutoff,
                )
                if match:
                    location = [i for i, x in enumerate(df_sheet.iloc[row] == match[0][0]) if x]
                    result = df_sheet.iloc[row][location[0]+1]
                    dictionary[name] = result
        df = df.append(dictionary, ignore_index=True)
df.tail()